# Test the thesis BERT model

First thing is first let's import some modules that we will need.

In [43]:
!pip install tensorboard

In [1]:
!pip install tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.6 MB/s eta 0:00:00


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00


In [18]:
import os
import sys
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from transformers import RobertaTokenizer, pipeline
from transformers import RobertaConfig, RobertaForMaskedLM, AdamW
from torch.utils.tensorboard import SummaryWriter
import torch
import tqdm

print(sys.path)
print(os.getcwd())

['/content/bert-transformer', '/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']
/content


Now we must mount google drive and then clone the repo containing the code.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!git clone https://github.com/christianbromley/bert-transformer.git

Cloning into 'bert-transformer'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 45 (delta 21), reused 35 (delta 14), pack-reused 0
Unpacking objects: 100% (45/45), 133.45 KiB | 579.00 KiB/s, done.


In [2]:
sys.path.insert(0, '/content/bert-transformer')

from src.make_tokens import extract_file_paths, run as run_tokenizer

Next get the path to our text.

In [8]:
DATA_DIR='/content/bert-transformer/data'

paths = extract_file_paths(DATA_DIR)

print(paths)

['/content/bert-transformer/data/text_0.txt']


In [4]:
TOKENIZER_OUTPUT_DIR = '/content/bert-transformer/models'

run_tokenizer(paths, TOKENIZER_OUTPUT_DIR)

Ok, so we have run our tokenizer and produced the files now contained in the models directory.

Now we must initialise our tokenizer.

In [5]:
# initialize the tokenizer using the tokenizer we created
tokenizer = RobertaTokenizer.from_pretrained(TOKENIZER_OUTPUT_DIR, max_len=512)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


In [47]:
tokenizer('This is a cytokine')

{'input_ids': [0, 56, 76, 77, 87, 225, 77, 87, 225, 69, 225, 71, 93, 88, 83, 79, 77, 82, 73, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Ok, so we have our tokenizer. Now we want to create our tensors. First we need  a function to create our masked language model.

In [6]:
def masked_language_model(tensor):
    rand = torch.rand(tensor.shape)
    mask_arr = (rand < 0.15) * (tensor > 2)
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i,selection] = 4
    return tensor

In [9]:
## input ids - token IDs with a % of tokens masked with the mask token ID which in our case is 4
input_ids = []
## mask - this is a binary tensor of 1 and 0 indicating where the masks are
mask = []
## labels - these are just the unmasked token IDs
labels = []
# initialise the output
tokenized_text = {}

# read file
with open(paths[0], 'r', encoding='utf-8') as fp:
    lines = fp.read().split('\n')

# get the file name
fname = paths[0].split('/')[-1]
print(fname)

text_0.txt


In [10]:
# tokenize the text in these lines
sample = tokenizer(lines, max_length=512, padding='max_length', truncation=True, return_tensors='pt')

tokenized_text[fname] = sample

# the sample object contains some of our tensors - extract these

## get the input IDs and append to labels
labels.append(sample.input_ids)

## get the attention mask - the binary
mask.append(sample.attention_mask)


In [9]:
#print(sample.keys())

In [10]:
#print(sample.attention_mask[0])

In [11]:
## now apply the masked language model function on the input IDs to mask 15% of tokens
mlm_on_input = masked_language_model(sample.input_ids.detach().clone())
input_ids.append(mlm_on_input)

In [12]:
# now format tensors
input_ids = torch.cat(input_ids)
mask = torch.cat(mask)
labels = torch.cat(labels)

In [13]:
# create encodings dict
encodings = {
    'input_ids': input_ids,
    'mask': mask,
    'labels': labels
}

In [14]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return self.encodings['input_ids'].shape[0]
    def __getitem__(self, i):
        # return {
        #     'input_ids': self.encodings['input_ids'][i]
        # }
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [15]:
dataset = Dataset(encodings)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [16]:
# build RoBERTa config
config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)


In [34]:
# init tensorboard writer
writer = SummaryWriter()
# init model
model = RobertaForMaskedLM(config)
# set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)
# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)
# set number of epochs
epochs = 2
# loop through epochs
for epoch in range(epochs):
    print(f'Epoch {str(epoch)}')
    # setup loop with TQDM and dataloader
    loop = tqdm.tqdm(dataloader, leave=False)
    for batch_idx, batch in enumerate(loop):
        #print(batch.keys())
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        #print(batch.keys())
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['mask'].to(device)
        labels = batch['labels'].to(device)
        # model
        outputs = model(input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
        # extract the loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

        writer.add_scalar('Loss', loss.item(), epoch * len(dataloader) + batch_idx)
writer.flush()
writer.close()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0


Epoch 1


In [38]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [45]:
!tensorboard --logdir=runs --port=8008 --bind_all

2023-07-18 22:12:39.131137: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server)
/usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server)
/usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server)
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.12.3 at http://localhost:8008/ (Press CTRL+C to quit)
^C


In [35]:
model.save_pretrained(f'/content/bert-transformer/models/thesis_bert')

In [48]:
my_pl = pipeline('fill-mask',
                 model='/content/bert-transformer/models/thesis_bert',
                 tokenizer=tokenizer)

In [51]:
my_pl(f'Cancer is a {my_pl.tokenizer.mask_token}')

[{'score': 0.1518344134092331,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Cancer is a '},
 {'score': 0.087583027780056,
  'token': 88,
  'token_str': 't',
  'sequence': 'Cancer is at'},
 {'score': 0.06038049980998039,
  'token': 73,
  'token_str': 'e',
  'sequence': 'Cancer is ae'},
 {'score': 0.04791000485420227,
  'token': 83,
  'token_str': 'o',
  'sequence': 'Cancer is ao'},
 {'score': 0.0468127578496933,
  'token': 82,
  'token_str': 'n',
  'sequence': 'Cancer is an'}]

In [50]:
my_pl(f'Inflammation promotes {my_pl.tokenizer.mask_token}')

[{'score': 0.17040212452411652,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Inflammation promotes '},
 {'score': 0.07921242713928223,
  'token': 88,
  'token_str': 't',
  'sequence': 'Inflammation promotest'},
 {'score': 0.055338140577077866,
  'token': 73,
  'token_str': 'e',
  'sequence': 'Inflammation promotese'},
 {'score': 0.050345294177532196,
  'token': 77,
  'token_str': 'i',
  'sequence': 'Inflammation promotesi'},
 {'score': 0.049357637763023376,
  'token': 82,
  'token_str': 'n',
  'sequence': 'Inflammation promotesn'}]

In [22]:
def train_model(config, loader_for_data):
    # init model
    model = RobertaForMaskedLM(config)
    # set device
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    # and move our model over to the selected device
    model.to(device)
    # activate training mode
    model.train()
    # initialize optimizer
    optim = AdamW(model.parameters(), lr=1e-4)
    # set number of epochs
    epochs = 2
    # loop through epochs
    for epoch in range(epochs):
        print(f'Epoch {str(epoch)}')
        # setup loop with TQDM and dataloader
        loop = tqdm.tqdm(loader_for_data, leave=False)
        for batch in loop:
            print(batch.keys())
            # initialize calculated gradients (from prev step)
            optim.zero_grad()
            # pull all tensor batches required for training
            #print(batch.keys())
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['mask'].to(device)
            labels = batch['labels'].to(device)
            # model
            outputs = model(input_ids,
                            attention_mask=attention_mask,
                            labels=labels)
            # extract the loss
            loss = outputs.loss
            # calculate loss for every parameter that needs grad update
            loss.backward()
            # update parameters
            optim.step()
            # print relevant info to progress bar
            loop.set_description(f'Epoch {epoch}')
            loop.set_postfix(loss=loss.item())

    return model



In [24]:
model = train_model(config=config, loader_for_data=dataloader)

model.save_pretrained(f'/content/bert-transformer/models/thesis_bert')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0



  0%|          | 0/117 [00:00<?, ?it/s]

dict_keys(['input_ids', 'mask', 'labels'])



Epoch 0:   1%|          | 1/117 [00:01<02:19,  1.20s/it, loss=10.5]

...

dict_keys(['input_ids', 'mask', 'labels'])



Epoch 1:  99%|█████████▉| 116/117 [02:14<00:01,  1.16s/it, loss=0.157]

dict_keys(['input_ids', 'mask', 'labels'])



Epoch 1: 100%|██████████| 117/117 [02:15<00:00,  1.08s/it, loss=0.177]
                                                                      

AttributeError: ignored

In [26]:
print(model)

None


I am up to here

In [12]:
tokenized_text, encodings = build_tensors(paths, tokenizer)

[tensor([[   0, 1202,   37,  ...,    1,    1,    1],
        [   0,    4,  309,  ...,    1,    1,    1],
        [   0,  892,  568,  ...,    1,    1,    1],
        ...,
        [   0, 1543,  261,  ...,    1,    1,    1],
        [   0, 3796,   93,  ...,    1,    1,    1],
        [   0,  564,    4,  ...,    1,    1,    1]])]


Within encodings we should have our three tensors:
1. input_ids - token IDs with a % of tokens masked with the mask token ID which in our case is 4
2. mask - this is a binary tensor of 1 and 0 indicating where the masks are
3. labels - these are just the unmasked token IDs

In [13]:
encodings.keys()

dict_keys(['input_ids', 'mask', 'labels'])

In [14]:
# create the data loader with a batch size of 16
# this function uutilises the torch data loader
loader = create_data_loader(encodings)

In [17]:
# build RoBERTa config
config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)


Now actually train the model

In [18]:
train_model(config, loader)

model.save_pretrained(f'{output_dir}/thesis_bert')

Epoch 0


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/117 [00:00<?, ?it/s]

dict_keys(['input_ids'])


KeyError: ignored

In [ ]:

import argparse
from pathlib import Path

#sys.path.insert(0,'./.env/lib/python3.8/site-packages')
from tokenizers import ByteLevelBPETokenizer
#from transformers import RobertaTokenizer
import os

from transformers import RobertaTokenizer, RobertaConfig, RobertaForMaskedLM, AdamW
import torch
import tqdm

/Users/christianbromley/.virtualenvs/machine-learning-course/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# constants
PDF_PATH='/Users/christianbromley/Documents/Personal/PhD/Christian_Bromley_Final_Thesis_20211221_cover.pdf'
OUTPUT_PATH='/Users/christianbromley/Documents/Personal/PhD/Christian_Bromley_Final_Thesis_20211221.csv'
TEXT_PARSE_DIR='/Users/christianbromley/Documents/Personal/PhD/thesis_parsed'
MODELS_DIR='../models'

In [ ]:
def extract_file_paths(input_dir):
    paths = [str(x) for x in Path(input_dir).glob('**/*.txt')]
    return paths

def run(paths, output_dir):
    # intialise tokeniser
    tokenizer = ByteLevelBPETokenizer()
    # train
    tokenizer.train(files=paths,
                    vocab_size=30_522,
                    min_frequency=2,
                    special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

    tokenizer.save_model(output_dir)

In [ ]:
paths = extract_file_paths(input_dir=TEXT_PARSE_DIR)

paths

['/Users/christianbromley/Documents/Personal/PhD/thesis_parsed/text_0.txt']

In [ ]:
run(paths, MODELS_DIR)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


In [ ]:
def masked_language_model(tensor):
    rand = torch.rand(tensor.shape)
    mask_arr = (rand < 0.15) * (tensor > 2)
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i,selection] = 4
    return tensor

In [19]:
## input ids - token IDs with a % of tokens masked with the mask token ID which in our case is 4
input_ids = []
## mask - this is a binary tensor of 1 and 0 indicating where the masks are
mask = []
## labels - these are just the unmasked token IDs
labels = []
# initialise the output
tokenized_text = {}

In [20]:
# read file
with open(paths[0], 'r', encoding='utf-8') as fp:
    lines = fp.read().split('\n')

In [ ]:
lines

In [21]:
# get the file name
fname = paths[0].split('/')[-1]
fname

'text_0.txt'

In [22]:
# tokenize the text in these lines
sample = tokenizer(lines, max_length=512, padding='max_length', truncation=True, return_tensors='pt')

In [23]:
sample.keys()

dict_keys(['input_ids', 'attention_mask'])

In [24]:
tokenized_text[fname] = sample

In [25]:
# the sample object contains some of our tensors - extract these
## get the input IDs and append to labels
labels.append(sample.input_ids)
## get the attention mask - the binary
mask.append(sample.attention_mask)


In [26]:
sample.input_ids.detach().clone()

tensor([[   0, 1202,   37,  ...,    1,    1,    1],
        [   0,   56,  309,  ...,    1,    1,    1],
        [   0,  892,  568,  ...,    1,    1,    1],
        ...,
        [   0, 1543,  261,  ...,    1,    1,    1],
        [   0, 3796,   93,  ...,    1,    1,    1],
        [   0,  564, 2787,  ...,    1,    1,    1]])

In [27]:
## now apply the masked language model function on the input IDs to mask 15% of tokens
mlm_on_input = masked_language_model(sample.input_ids.detach().clone())
input_ids.append(mlm_on_input)

In [28]:
# construct the output
encodings = {
    'input_ids': input_ids[0],
    'mask': mask[0],
    'labels': labels[0]
}

In [29]:
len(encodings['labels'])

1868

In [31]:
import torch

In [32]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return self.encodings['input_ids'].shape[0]
    def __getitem__(self, i):
        return {
            'input_ids': self.encodings['input_ids'][i]
        }

In [ ]:
# def create_data_loader(encodings):
#     dataset = Dataset(encodings)
#     data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)
#     return data_loader

In [33]:
loader = create_data_loader(encodings)

In [34]:
loader.dataset.encodings

{'input_ids': tensor([[   0,    4,   37,  ...,    1,    1,    1],
         [   0,   56,  309,  ...,    1,    1,    1],
         [   0,  892,  568,  ...,    1,    1,    1],
         ...,
         [   0, 1543,    4,  ...,    1,    1,    1],
         [   0, 3796,   93,  ...,    1,    1,    1],
         [   0,  564, 2787,  ...,    1,    1,    1]]),
 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[   0, 1202,   37,  ...,    1,    1,    1],
         [   0,   56,  309,  ...,    1,    1,    1],
         [   0,  892,  568,  ...,    1,    1,    1],
         ...,
         [   0, 1543,  261,  ...,    1,    1,    1],
         [   0, 3796,   93,  ...,    1,    1,    1],
         [   0,  564, 2787,  ...,    1,    1,    1]])}

In [35]:
# build RoBERTa config
config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)


# init model
model = RobertaForMaskedLM(config)
# set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)
# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [36]:
# set number of epochs
epochs = 2

In [ ]:
for epoch in range(epochs):
    input_ids = loader.dataset.encodings['input_ids'].to(device)
    attention_mask = loader.dataset.encodings['mask'].to(device)
    labels = loader.dataset.encodings['labels'].to(device)
    print('go')
    # model
    outputs = model(input_ids,
                    attention_mask=attention_mask,
                    labels=labels)
    # extract the loss
    loss = outputs.loss
    # calculate loss for every parameter that needs grad update
    loss.backward()
    # update parameters
    optim.step()
    # print relevant info to progress bar
    loop.set_description(f'Epoch {epoch}')
    loop.set_postfix(loss=loss.item())


go


In [ ]:
#loop = tqdm(loader, leave=True)
# loop through epochs
for epoch in range(epochs):
    print(f'Epoch {str(epoch)}')
    for batch, feed_dict in tqdm.tqdm(enumerate(loader)):
        print(i_batch)
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        #print(batch.keys())
        input_ids = batch['input_ids'][0].to(device)
        attention_mask = batch['mask'][0].to(device)
        labels = batch['labels'][0].to(device)
        # model
        outputs = model(input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
        # extract the loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

model.save_pretrained(f'{output_dir}/thesis_bert')

Epoch 0


0it [00:00, ?it/s]

116


AttributeError: 'int' object has no attribute 'keys'

In [ ]:
# loop through epochs
for epoch in range(epochs):
    print(f'Epoch {str(epoch)}')
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    print(loop)
    for batch in loop:
        print(batch.keys())
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        print(batch.keys())
        input_ids = batch['input_ids'][0].to(device)
        attention_mask = batch['mask'][0].to(device)
        labels = batch['labels'][0].to(device)
        # model
        outputs = model(input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
        # extract the loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

model.save_pretrained(f'{output_dir}/thesis_bert')

Epoch 0


TypeError: 'module' object is not callable

In [ ]:
def train_model(config, loader):
    # init model
    model = RobertaForMaskedLM(config)
    # set device
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    # and move our model over to the selected device
    model.to(device)
    # activate training mode
    model.train()
    # initialize optimizer
    optim = AdamW(model.parameters(), lr=1e-4)
    # set number of epochs
    epochs = 2
    # loop through epochs
    for epoch in range(epochs):
        print(f'Epoch {str(epoch)}')
        # setup loop with TQDM and dataloader
        loop = tqdm(loader, leave=True)
        for batch in loop:
            print(batch.keys())
            # initialize calculated gradients (from prev step)
            optim.zero_grad()
            # pull all tensor batches required for training
            print(batch.keys())
            input_ids = batch['input_ids'][0].to(device)
            attention_mask = batch['mask'][0].to(device)
            labels = batch['labels'][0].to(device)
            # model
            outputs = model(input_ids,
                            attention_mask=attention_mask,
                            labels=labels)
            # extract the loss
            loss = outputs.loss
            # calculate loss for every parameter that needs grad update
            loss.backward()
            # update parameters
            optim.step()
            # print relevant info to progress bar
            loop.set_description(f'Epoch {epoch}')
            loop.set_postfix(loss=loss.item())

In [ ]:

train_model(config, loader)

model.save_pretrained(f'{output_dir}/thesis_bert')

Epoch 0


  0%|                                                                                                                                                                                                           | 0/117 [00:00<?, ?it/s]

dict_keys(['input_ids'])
dict_keys(['input_ids'])


KeyError: 'mask'